# CSIRO Competition Solution Notebook

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import sys
sys.path.append("/kaggle/input/sam-optim")

In [ ]:
import torch

torch.cuda.is_available()

In [ ]:
from sam import *

In [ ]:
# # Visualize target distributions
# fig, axes = plt.subplots(2, 3, figsize=(18, 10))
# axes = axes.ravel()

# target_names = df['target_name'].unique()
# for idx, target in enumerate(target_names[:6]):
#     data = df[df['target_name'] == target]['target']
#     axes[idx].hist(data, bins=50, edgecolor='black', alpha=0.7)
#     axes[idx].set_title(f'{target}\nMean: {data.mean():.2f}, Std: {data.std():.2f}')
#     axes[idx].set_xlabel('Target Value (g)')
#     axes[idx].set_ylabel('Frequency')
# plt.tight_layout()
# plt.show()

In [ ]:
# Data Transform

from torchvision.transforms import v2
import torch

# to_tensor = v2.ToTensor()
# img_tensor = to_tensor(img)

dtype = torch.float32
img_size = (224, 224)
image_transform = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(dtype, scale=True),    
    v2.Resize(img_size),
    v2.RandomHorizontalFlip(p=0.5),
    v2.RandomVerticalFlip(p=0.5),
    v2.RandomRotation(5, interpolation=v2.InterpolationMode.BILINEAR),
    v2.ColorJitter(
        brightness=0.25,
        contrast=0.25,
        saturation=0.25,
        hue=0.05,
    ),
    # v2.RandomAdjustSharps
    v2.Normalize(mean=[0.485, 0.456, 0.406],
                 std=[0.229, 0.224, 0.225]),
])

val_transform = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(dtype, scale=True),
    v2.Resize(img_size),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


def numeric_transform(X, X_max, X_min) -> torch.Tensor:
    X_normalized = (X - X_min) / (X_max - X_min)
    return X_normalized

def target_transform(targets) -> torch.Tensor:
    return torch.log1p(targets)

def target_untransform(targets) -> torch.Tensor:
    return torch.expm1(targets)

def categorical_transform(row) -> torch.Tensor:
    return row

# Train Set

In [ ]:

from torch.utils.data import Dataset
from torchvision.io import decode_image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import pandas as pd

class Image2BioMassTrainValDataset(Dataset):
    
    def __init__(self, dataset_path, img_transform=None, numeric_transform=None,categorical_transform=None, target_transform=None):
        
        self.df = self.process_df(dataset_path)
        self.dataset_path = dataset_path
        self.img_transform = img_transform
        self.target_transform = target_transform
        self.numeric_transform = numeric_transform
        self.categorical_transform = categorical_transform
        self.targets = self.df.loc[:, ["Dry_Green_g", "Dry_Dead_g", "Dry_Clover_g"]]


    def process_df(self, dataset_path):
        self.le_date = LabelEncoder()
        self.le_state = LabelEncoder()
        self.le_species = LabelEncoder()

        df = pd.read_csv(os.path.join(dataset_path, "train.csv"))
        df['base_sample_id'] = df['sample_id'].str.split('__').str[0]
        df = df.pivot_table(
        index=['base_sample_id', 'image_path', 'Sampling_Date', 'State', 'Species', 'Pre_GSHH_NDVI', 'Height_Ave_cm'],
        columns='target_name',
        values='target'
        ).reset_index()
        df["Sampling_Date"] = self.le_date.fit_transform(df["Sampling_Date"])
        df["State"] = self.le_state.fit_transform(df["State"])
        df["Species"] = self.le_species.fit_transform(df["Species"])
        # display(df)
        return df

    def __len__(self):
        return len(self.df)

    def get_cat_features(self):
        return ["Sampling_Date", "State", "Species"]
    
    def get_cat_vocab_sizes(self):
        results = []

        for i in self.get_cat_features():
            results.append(len(self.df[i].unique()))
        return results

    def __getitem__(self, idx):
        # B = batch_size
        # display(self.df)
        img_path = os.path.join(self.dataset_path, self.df.loc[idx, 'image_path'])
        image = decode_image(img_path)
        # display(self.df)
        numeric_features = torch.tensor([
            self.df.loc[idx, "Pre_GSHH_NDVI"],
            self.df.loc[idx, "Height_Ave_cm"],
        ], dtype=torch.float32)

        categorical_features = torch.tensor([
            self.df.loc[idx, "Sampling_Date"],
            self.df.loc[idx, "State"],
            self.df.loc[idx, "Species"],
        ], dtype=torch.long)
        

        if self.img_transform:
            image = self.img_transform(image)
            
        if self.numeric_transform:
            # numeric_features[0] = self.numeric_transform(
            #     numeric_features[0],
            #     self.df.loc[:, "Pre_GSHH_NDVI"].max(), 
            #     self.df.loc[:, "Pre_GSHH_NDVI"].min()
            # )
            numeric_features[1] = self.numeric_transform(
                numeric_features[1], 
                self.df.loc[:, "Height_Ave_cm"].max(), 
                self.df.loc[:, "Height_Ave_cm"].min()
            )
            # print(numeric_features)
        combined_features = torch.cat([categorical_features.float(), numeric_features], dim=0)
        # print(combined_features)
        targets = torch.Tensor(self.targets.iloc[idx].values)
        if self.target_transform:
            targets = self.target_transform(targets)
        return image, combined_features, targets

# Test Set

In [ ]:

# from torch.utils.data import Dataset
# from torchvision.io import decode_image
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import train_test_split
# from torch.utils.data import DataLoader
# import pandas as pd

# class Image2BioMassTestFromTrainDataset(Dataset):
    
#     def __init__(self, dataset_path, img_transform=None, numeric_transform=None,categorical_transform=None):
        
#         self.df = self.process_df(dataset_path)
#         self.dataset_path = dataset_path
#         self.img_transform = img_transform
#         self.numeric_transform = numeric_transform
#         self.categorical_transform = categorical_transform

#     def process_df(self, dataset_path):
#         self.le_date = LabelEncoder()
#         self.le_state = LabelEncoder()
#         self.le_species = LabelEncoder()

#         df = pd.read_csv(os.path.join(dataset_path, "train.csv"))
#         df['base_sample_id'] = df['sample_id'].str.split('__').str[0]
#         df = (
#             df.assign(_val="")
#               .pivot(index=['base_sample_id', "image_path"],
#                      columns='target_name',
#                      values='_val')
#               .reset_index()
#         )

#         return df

#     def __len__(self):
#         return len(self.df)

#     def get_cat_features(self):
#         return ["Sampling_Date", "State", "Species"]
    
#     def get_cat_vocab_sizes(self):
#         results = []

#         for i in self.get_cat_features():
#             results.append(len(self.df[i].unique()))
#         return results

#     def __getitem__(self, idx):

#         img_path = os.path.join(self.dataset_path, self.df.loc[idx, 'image_path'])
#         image = decode_image(img_path)

#         # Use val_transform for test data (no augmentation)
#         if self.img_transform:
#             image = self.img_transform(image)
#         else:
#             # Fallback basic transform if no transform provided
#             transform = v2.Compose([
#                 v2.ToImage(),
#                 v2.ToDtype(dtype, scale=True),
#                 v2.Resize((518, 518)),
#                 v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#             ])
#             image = transform(image)

#         combined_features = torch.zeros(5, dtype=torch.float32)
#         sample_id = self.df.loc[idx, 'base_sample_id']
#         return image, combined_features, sample_id

In [ ]:

from torch.utils.data import Dataset
from torchvision.io import decode_image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import pandas as pd

class Image2BioMassTestDataset(Dataset):
    
    def __init__(self, dataset_path, img_transform=None, numeric_transform=None,categorical_transform=None):
        
        self.df = self.process_df(dataset_path)
        self.dataset_path = dataset_path
        self.img_transform = img_transform
        self.numeric_transform = numeric_transform
        self.categorical_transform = categorical_transform

    def process_df(self, dataset_path):
        self.le_date = LabelEncoder()
        self.le_state = LabelEncoder()
        self.le_species = LabelEncoder()

        df = pd.read_csv(os.path.join(dataset_path, "test.csv"))
        df['base_sample_id'] = df['sample_id'].str.split('__').str[0]
        df = (
            df.assign(_val="")
              .pivot(index=['base_sample_id', "image_path"],
                     columns='target_name',
                     values='_val')
              .reset_index()
        )

        return df

    def __len__(self):
        return len(self.df)

    def get_cat_features(self):
        return ["Sampling_Date", "State", "Species"]
    
    def get_cat_vocab_sizes(self):
        results = []

        for i in self.get_cat_features():
            results.append(len(self.df[i].unique()))
        return results

    def __getitem__(self, idx):

        img_path = os.path.join(self.dataset_path, self.df.loc[idx, 'image_path'])
        image = decode_image(img_path)

        # Use val_transform for test data (no augmentation)
        if self.img_transform:
            image = self.img_transform(image)
        else:
            # Fallback basic transform if no transform provided
            transform = v2.Compose([
                v2.ToImage(),
                v2.ToDtype(dtype, scale=True),
                v2.Resize((518, 518)),
                v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])
            image = transform(image)

        combined_features = torch.zeros(5, dtype=torch.float32)
        sample_id = self.df.loc[idx, 'base_sample_id']
        return image, combined_features, sample_id

In [ ]:
test_dataset = Image2BioMassTestDataset(
    dataset_path="/kaggle/input/csiro-biomass/",
    img_transform=val_transform,  # Use val_transform (no augmentation, proper size)
    
)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)
next(iter(test_dataloader))[2]

# Train Split

In [ ]:

from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset

# Create base dataset to get indices
base_dataset = Image2BioMassTrainValDataset(
    dataset_path="/kaggle/input/csiro-biomass/",
    img_transform=None,  # no transform yet
    numeric_transform=numeric_transform,
    target_transform=target_transform
)

# Split indices
seed = 42
train_indices, val_indices = train_test_split(
    range(len(base_dataset)), 
    train_size=0.8, 
    shuffle=True, 
    random_state=seed
)

# Create training dataset WITH augmentation
train_dataset = Image2BioMassTrainValDataset(
    dataset_path="/kaggle/input/csiro-biomass/",
    img_transform=image_transform,  # WITH augmentation
    numeric_transform=numeric_transform,
    target_transform=target_transform
)
train_dataset = Subset(train_dataset, train_indices)

val_dataset = Image2BioMassTrainValDataset(
    dataset_path="/kaggle/input/csiro-biomass/",
    img_transform=val_transform,  # WITHOUT augmentation
    numeric_transform=numeric_transform,
    target_transform=target_transform
)
val_dataset = Subset(val_dataset, val_indices)

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [ ]:
print("img features:", next(iter(train_dataloader))[0].shape)
print("tabular features:", next(iter(train_dataloader))[1][0])
print("Dry Clover, Dry Dead, Dry Green:", next(iter(train_dataloader))[2][0])

# Model

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

BATCH_SIZE=32
HEIGHT=224
WIDTH=224
NUM_CHANNELS=3
class BackBone(nn.Module):

    def __init__(self, num_channels=3):
        super(BackBone, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=num_channels, out_channels=32, kernel_size=3, padding=1)
        self.batch_norm1 = nn.BatchNorm2d(32)
        self.activ1 = nn.GELU()
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=16, kernel_size=3, padding=1)
        self.batch_norm2 = nn.BatchNorm2d(16)
        self.activ2 = nn.GELU()
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=3, kernel_size=3, padding=1)
        self.batch_norm3 = nn.BatchNorm2d(3)
        self.activ3 = nn.GELU()

        self.downsample = None

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.batch_norm1(out)
        out = self.activ1(out)
        out = self.conv2(out)
        out = self.batch_norm2(out)
        out = self.activ2(out)
        out = self.conv3(out)
        out = self.batch_norm3(out)
        # print(out.shape)
        # print(identity.shape)
        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity

        return out

class Image2BiomassModel(nn.Module):

    def __init__(self):
        super(Image2BiomassModel, self).__init__()
        # self.backbone = Dinov2Model.from_pretrained(
        #     "/kaggle/input/dinov2/pytorch/base/1/"
        # )

        self.backbone = BackBone(num_channels=3)
        self.prelu = nn.PReLU()

        # ---- MLP Head ----
        self.noise = nn.Sequential(
            nn.AlphaDropout(0.1),
        )

        self.fc1 = nn.Sequential(
            nn.Linear(HEIGHT * WIDTH * NUM_CHANNELS, 512),
            nn.BatchNorm1d(512),
            nn.PReLU(),
            nn.Dropout(0.4),
        )

        self.fc2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.LayerNorm(256),
            nn.PReLU(),
            nn.Linear(256, 128),
            nn.LayerNorm(128),
            nn.PReLU(),
            nn.Dropout(0.4),
        )

        self.residual = nn.Sequential(
            nn.Linear(128, 128),
            nn.LayerNorm(128),
            nn.PReLU(),
            nn.Linear(128, 128),
            nn.LayerNorm(128),
        )
        self.out = nn.Linear(128, 3)

        self.criterion = nn.SmoothL1Loss(beta=0.5)
        
    def forward(self, x, y=None):
        # DINOv2-giant expects normalized images and outputs [B, 1536]
        outputs = self.backbone(x, )
        # x = outputs.last_hidden_state[:, 0]

        x = outputs.view(outputs.shape[0], -1)
        # print()
        x = self.noise(x)
        x = self.fc1(x)
        x = self.fc2(x)

        res = self.residual(x)
        x = x + res
        x = self.prelu(x)
        # x = F.Mish(x)

        preds = self.out(x)

        loss = None
        if y is not None:
            loss = self.criterion(preds, y)

        return preds, loss

sample = next(iter(train_dataloader))
model = Image2BiomassModel()

model(sample[0], sample[2])

# Train Loop

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Image2BiomassModel().to(device)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# base_optimizer = torch.optim.AdamW
# optimizer = SAM(model.parameters(), base_optimizer, lr=1e-4, weight_decay=1e-2)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-2)
weights = torch.tensor([0.1, 0.1, 0.1, 0.2, 0.5], device=device)

train_losses, val_losses = [], []
train_r2_history, val_r2_history = [], []


# eval metrics
def weighted_r2(y_true, y_pred, weights):
    mean = y_true.mean(dim=0)
    SSE = ((y_true - y_pred)**2).sum(dim=0)
    TSS = ((y_true - mean)**2).sum(dim=0)
    TSS = torch.clamp(TSS, min=1e-8)
    R2 = 1 - SSE / TSS
    R2 = torch.clamp(R2, min=-10, max=1)
    return (R2 * weights).sum() / weights.sum()

In [ ]:
from tqdm import tqdm
import torch
from torch.nn.utils import clip_grad_norm_

epochs = 100
for epoch in range(1, epochs+1):

    model.train()
    train_loss = 0
    train_r2_scores = []

    for imgs, _, y in tqdm(train_dataloader, desc=f"[Train] Epoch {epoch}"):

        imgs, y = imgs.to(device), y.to(device)

        # preds, loss = model(imgs, y)
        # optimizer.zero_grad()
        # print(loss.requires_grad)
        # def closure():
        #     # optimizer.zero_grad()
        #     loss.backward()
        #     return loss
        # # loss.backward()
        # optimizer.step(closure)
        
        # --- FIRST forward-backward (compute gradients on current weights) ---
        preds, loss = model(imgs, y)

        # ----- L1 regularization -----
        l1_lambda = 1e-6   # YOU set this value; this is a reasonable starting point
        reg_loss = sum(param.abs().sum() for param in model.parameters())
        loss = loss + l1_lambda * reg_loss

        optimizer.zero_grad()
        loss.backward()
        # perform SAM first step (perturb weights)
        
        clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        # optimizer.first_step(zero_grad=True)

        # # --- SECOND forward-backward on perturbed weights ---
        # # recompute preds & loss with perturbed weights, then backprop and second step
        # preds_2, loss_2 = model(imgs, y)

        # loss_2.backward()
        # optimizer.second_step(zero_grad=True)

        train_loss += loss.item()

        # ----- Add 2 more outputs (not for training) -----
        # Order: [Dry_Green_g, Dry_Dead_g, Dry_Clover_g, GDM_g, Dry_Total_g]
        preds5 = torch.cat([
            preds,
            (preds[:, 2] + preds[:, 0]).unsqueeze(1),  # GDM_g = Dry_Clover + Dry_Green
            preds.sum(dim=1).unsqueeze(1)             # Dry_Total_g = all three
        ], dim=1)

        y5 = torch.cat([
            y,
            (y[:, 2] + y[:, 0]).unsqueeze(1),  # GDM_g = Dry_Clover + Dry_Green
            y.sum(dim=1).unsqueeze(1)          # Dry_Total_g = all three
        ], dim=1)

        train_r2_scores.append(weighted_r2(y5, preds5, weights).item())

    avg_train_loss = train_loss / len(train_dataloader)
    avg_train_r2 = sum(train_r2_scores) / len(train_r2_scores)


    # ---------------- Validation ----------------
    model.eval()
    val_loss = 0
    val_r2_scores = []

    with torch.no_grad():
        for imgs, _, y in tqdm(val_dataloader, desc=f"[Val] Epoch {epoch}"):

            imgs, y = imgs.to(device), y.to(device)

            preds, loss = model(imgs, y)
            val_loss += loss.item()

            preds5 = torch.cat([
                preds,
                (preds[:, 2] + preds[:, 0]).unsqueeze(1),  # GDM_g = Dry_Clover + Dry_Green
                preds.sum(dim=1).unsqueeze(1)             # Dry_Total_g = all three
            ], dim=1)

            y5 = torch.cat([
                y,
                (y[:, 2] + y[:, 0]).unsqueeze(1),  # GDM_g = Dry_Clover + Dry_Green
                y.sum(dim=1).unsqueeze(1)          # Dry_Total_g = all three
            ], dim=1)

            val_r2_scores.append(weighted_r2(y5, preds5, weights).item())

    avg_val_loss = val_loss / len(val_dataloader)
    avg_val_r2 = sum(val_r2_scores) / len(val_r2_scores)
    val_losses.append(avg_val_loss)
    train_losses.append(avg_train_loss)
    val_r2_history.append(avg_val_r2)
    train_r2_history.append(avg_train_r2)

    print(f"Epoch {epoch} | Train Loss: {avg_train_loss:.4f} | "
          f"Train R2: {avg_train_r2:.4f} | Val Loss: {avg_val_loss:.4f} | Val R2: {avg_val_r2:.4f}")

In [ ]:
import matplotlib.pyplot as plt

# ---------------------------
# LOSS PLOTS
# ---------------------------
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training vs Validation Loss")
plt.legend()
plt.grid(True)
plt.show()

# ---------------------------
# R² PLOTS
# ---------------------------
plt.figure(figsize=(10, 5))
plt.plot(train_r2_history, label="Train R2")
plt.plot(val_r2_history, label="Validation R2")
plt.xlabel("Epoch")
plt.ylabel("R² Score")
plt.title("Training vs Validation R²")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
import torch
import pandas as pd
from tqdm import tqdm

model.eval()
rows = []

target_cols = [
    "Dry_Green_g",
    "Dry_Dead_g",
    "Dry_Clover_g",
    "GDM_g",
    "Dry_Total_g",
]

test_dataset = Image2BioMassTestDataset(
    dataset_path="/kaggle/input/csiro-biomass/",
    # img_transform=image_transform,
)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

with torch.no_grad():
    for imgs, _, sample_ids in tqdm(test_dataloader, desc="Inference"):
        
        imgs = imgs.to(device)

        # (B, 3) - model outputs: [Dry_Green_g, Dry_Dead_g, Dry_Clover_g]
        preds3, _ = model(imgs, y=None)

        # reverse log transform -> still (B,3)
        # print(preds3)
        preds3 = target_untransform(preds3).cpu().numpy()

        # extract 3 predictions in the correct order
        dg = preds3[:, 0]  # Dry_Green_g
        dd = preds3[:, 1]  # Dry_Dead_g
        dc = preds3[:, 2]  # Dry_Clover_g

        # compute extra targets
        gdm = dg + dc
        dry_total = dg + dd + dc

        preds5 = np.stack([dg, dd, dc, gdm, dry_total], axis=1)
        np.set_printoptions(suppress=True, precision=4)
        # print(preds5)

        # build submission rows
        for sid, pred_vec in zip(sample_ids, preds5):
            for col, value in zip(target_cols, pred_vec):
                rows.append({
                    "sample_id": f"{sid}__{col}",
                    "target": float(value)
                })

df_submit = pd.DataFrame(rows)
df_submit.to_csv("submission.csv", index=False)
print("Saved submission.csv")
df_submit.head(20)